# Rafael Nunes Santana - 201800215

### Atividade 9 (Teste 2) - Pesquisa Operacional I

Nessa atividade estaremos implementando o algoritmo Simplex de duas fases com base no código feito para a atividade 8. Esse algoritmo trabalha com variáveis positivas, reais e restrições do tipo igualdade (A(i,:)*x = b(i)), de limite superior (A(i,:)*x <= b(i)) e de limite inferior (A(i,:)*x >= b(i)). Para tal estaremos trabalhando em cima do mesmo código entregue na parte 8.

Esse algoritmo se faz necessário quando não podemos assumir a solução inicial como x1 = x2 = ... = xn = 0. Então separamos o algoritmo em duas partes, onde na primeira estaremos apenas encontrando uma solução inicial possível para a segunda parte.

## Funcionamento do algoritmo

Na primeira fase, nós primeiramente devemos passar o problema para a forma padrão padronizando a função objetico e as restrições montando o tableau e adicinoando variáveis artificiais. Então criamos uma nova função objetivo, que busca minimizar as variáveis artificiais e tentamos resolver esse novo problema. Quando ele estier resolvido, teremos encontrado uma solução possível para o problema original se ele tiver solução (o que é indicado caso o resultado da minimização tenha sido 0). Caso o problema não tenha solução, abortamos o algoritmo.

Na segunda fase, pegamos o tableau resultante da primeira fase e focamos em maximizar ou minimizar a função objetico original. Como agora temos uma solução inicial possível, prosseguimos com o algoritmo normalmente e no fim teremos encontrado uma solução ótima.

## Alterando para minimização

Para que o algoritmo minimize a função objetiva, devemos escolher a variável mais positiva (ao invés da mais negativa) ao escolher a variável que entra. Além disso devemos alterar a condição de parada: ao invés de parar quando todos os coeficientes da função objetivo forem maiores ou iguais a zero, paramos quando eles forem todos menores ou iguais a zero.

In [4]:
import numpy as np

In [55]:
# Argumentos da função:
# c: vetor com os coeficientes da função objetivo
# A: matriz com os coeficientes das restrições
# b: vetor com os lados direito das restrições
# ctype: o tipo das comparações. S: igualdade, U: A(i,:)*x <= b(i), L: A(i,:)*x >= b(i)
# minimizando: Se a função objetiva deve ser minimizada. As restrições passam a ser do tipo A(i,:)*x <= b(i)

# OBS.: estamos considerando todas as variáveis >= 0
def simplex(c, A, b, ctype, maximizando=True):
    n_variaveis = len(c)
    n_restricoes = len(A)

    # adicionando as variáveis de folga:
    n_variaveis_folga = len(list(filter(lambda t: t != 'S', ctype)))
    
    for i in range(0, len(A)):
        for j in range(0, n_variaveis_folga):
            A[i].append(0)

    count = 0
    for i in range(0, len(A)):
        if ctype[i] == 'S':
            continue

        A[i][n_variaveis + count] = 1 if ctype[i] == 'U' else -1
        count += 1

    # adicionando as variáveis artificiais:
    n_variaveis_art = 0

    for i in range(0, len(A)):
        if (ctype[i] == 'U' and b[i] < 0) or \
        (ctype[i] == 'L' and b[i] > 0) or \
        (ctype[i] == 'S' and b[i] != 0):
            n_variaveis_art += 1
            for j in range(0, len(A)):
                A[j].append(1 if i == j else 0)

    n_variaveis = len(A[i])
    
    # criando e preenchendo a matriz tableau
    tableau = np.zeros((n_restricoes + 1, n_variaveis + 1), dtype=float)
    tableau[0, -n_variaveis_art - 1:-1] = [-1 for i in range(0, n_variaveis_art)]
    tableau[1:, :-1] = A
    tableau[1:, -1] = b
        
    print(tableau)
    return
    
    # Inicialemnte assumimos as variáveis não básicas ótima como [0, 0, ..., 0] e básicas como b
    valor_variaveis = np.zeros(len(c), dtype=float)
    valor_folga = np.array(b, dtype=float)
    
    # A condição de parada estão no fim da repetição. Ela muda caso estivermos maximizando ou minimizando
    while True:
        
        # Se estamos maximizando, escolhemos a menor variável e a maior caso estivermos minimizando
        variavel_extrema = 0

        if maximizando:
            variavel_extrema = min(tableau[0, 1:n_variaveis + 1])
        else:
            variavel_extrema = max(tableau[0, 1:n_variaveis + 1])
        
        indice_variavel_que_entra = list(tableau[0]).index(variavel_extrema)
        coeficientes_variavel_que_entra = tableau[1:, indice_variavel_que_entra]
   
        #print("indice elemento pivô:\n", indice_variavel_que_entra)
        #print("coeficientes pivô:\n", coeficientes_variavel_que_entra)
        
        termos_por_coeficientes = [tableau[1:, -1][i] / coeficientes_variavel_que_entra[i] for i in range(0, len(b))]
        # Ignoramos coeficientes negativos
        termos_por_coeficientes = list(filter(lambda i: i > 0, termos_por_coeficientes))
        indice_variavel_que_sai = 1
        menor_valor = termos_por_coeficientes[0]

        for i in range(1, len(termos_por_coeficientes)):
            if termos_por_coeficientes[i] < menor_valor:
                menor_valor = termos_por_coeficientes[i]
                indice_variavel_que_sai = i + 1
        
        elemento_pivo = tableau[indice_variavel_que_sai, indice_variavel_que_entra]
             
        #print("elemento pivo:\n", elemento_pivo)
        #print("linha pivô:\n", indice_variavel_que_sai)
        
        nova_linha_pivo = tableau[indice_variavel_que_sai] / elemento_pivo
        #print("nova linha pivô:\n", nova_linha_pivo)
        
        tableau[indice_variavel_que_sai] = nova_linha_pivo
        
        # Recalculando a tableau
        for i in range(0, len(tableau)):
            if i == indice_variavel_que_sai:
                continue
                
            coeficiente_variavel_que_entra = tableau[i, indice_variavel_que_entra]
            #print(coeficiente_variavel_que_entra)
            
            tableau[i] = nova_linha_pivo * (-coeficiente_variavel_que_entra) + tableau[i]
            
        print(tableau)

        # Se estamos maximizando, paramos quando todos os coeficientes forem maiores ou iguais a zero
        # Se estamos minimizando, paramos quando todos os coeficientes forem menores ou iguais a zero
        if maximizando and all([i >= 0 for i in tableau[0, 1:n_variaveis + 1]]):
            break
        elif not maximizando and all([i <= 0 for i in tableau[0, 1:n_variaveis + 1]]):
            break
        
    # coeficientes estão na ordem [x1, x2,..., xF1, xF2,...]
    coeficientes = []
    
    for j in range(1, n_variaveis + n_restricoes + 1):
        encontrou_valor = False

        for i in range(1, len(tableau)):
            if tableau[i, j] == 1:
                coeficientes.append(tableau[i, -1])
                encontrou_valor = True
                break
                
        if not encontrou_valor:
            coeficientes.append(0)
        
    return {'Z': tableau[0, -1], 'coeficientes': coeficientes}

In [56]:
A = [[3, 1],
     [4, 3],
     [1, 2]]

ctype = 'SLU'

simplex([3, 1], A, [3, 6, 4], ctype)

[[ 0.  0.  0.  0. -1. -1.  0.]
 [ 3.  1.  0.  0.  1.  0.  3.]
 [ 4.  3. -1.  0.  0.  1.  6.]
 [ 1.  2.  0.  1.  0.  0.  4.]]


In [58]:
A = [[ 1, 1],
     [-1, 1],
     [ 0, 1]]

ctype = 'LLU'

simplex([1, -2], A, [2, 1, 3], ctype)

[[ 0.  0.  0.  0.  0. -1. -1.  0.]
 [ 1.  1. -1.  0.  0.  1.  0.  2.]
 [-1.  1.  0. -1.  0.  0.  1.  1.]
 [ 0.  1.  0.  0.  1.  0.  0.  3.]]


In [3]:
# Exemplo maximização
A = [[1, 1,  1],
     [2, 1, -1],
     [3, 2, -1]]

simplex([2, 3, 1], A, [40, 20, 30])

[[ 1. -2. -3. -1.  0.  0.  0.  0.]
 [ 0.  1.  1.  1.  1.  0.  0. 40.]
 [ 0.  2.  1. -1.  0.  1.  0. 20.]
 [ 0.  3.  2. -1.  0.  0.  1. 30.]]
[[ 1.   2.5  0.  -2.5  0.   0.   1.5 45. ]
 [ 0.  -0.5  0.   1.5  1.   0.  -0.5 25. ]
 [ 0.   0.5  0.  -0.5  0.   1.  -0.5  5. ]
 [ 0.   1.5  1.  -0.5  0.   0.   0.5 15. ]]
[[ 1.          1.66666667  0.          0.          1.66666667  0.
   0.66666667 86.66666667]
 [ 0.         -0.33333333  0.          1.          0.66666667  0.
  -0.33333333 16.66666667]
 [ 0.          0.33333333  0.          0.          0.33333333  1.
  -0.66666667 13.33333333]
 [ 0.          1.33333333  1.          0.          0.33333333  0.
   0.33333333 23.33333333]]


{'Z': 86.66666666666667,
 'coeficientes': [0,
  23.333333333333336,
  16.666666666666668,
  0,
  13.333333333333334,
  0]}

In [4]:
# Exemplo minimização:
A = [[1,  2, 2,  4],
     [2, -1, 1,  2],
     [4, -2, 1, -1]]

simplex([5, -4, 6, -8], A, [40, 8, 10], False)

[[ 1. -5.  4. -6.  8.  0.  0.  0.  0.]
 [ 0.  1.  2.  2.  4.  1.  0.  0. 40.]
 [ 0.  2. -1.  1.  2.  0.  1.  0.  8.]
 [ 0.  4. -2.  1. -1.  0.  0.  1. 10.]]
[[  1.  -13.    8.  -10.    0.    0.   -4.    0.  -32. ]
 [  0.   -3.    4.    0.    0.    1.   -2.    0.   24. ]
 [  0.    1.   -0.5   0.5   1.    0.    0.5   0.    4. ]
 [  0.    5.   -2.5   1.5   0.    0.    0.5   1.   14. ]]
[[  1.     -7.      0.    -10.      0.     -2.      0.      0.    -80.   ]
 [  0.     -0.75    1.      0.      0.      0.25   -0.5     0.      6.   ]
 [  0.      0.625   0.      0.5     1.      0.125   0.25    0.      7.   ]
 [  0.      3.125   0.      1.5     0.      0.625  -0.75    1.     29.   ]]


{'Z': -80.0, 'coeficientes': [0, 6.0, 0, 7.0, 0, 0, 29.0]}

In [5]:
# Outro exemplo de minimização:
A = [[3, 2, 1],
     [2, 5, 3]]

simplex([-2, -3, -4], A, [10, 15], False)

[[ 1.  2.  3.  4.  0.  0.  0.]
 [ 0.  3.  2.  1.  1.  0. 10.]
 [ 0.  2.  5.  3.  0.  1. 15.]]
[[  1.          -0.66666667  -3.66666667   0.           0.
   -1.33333333 -20.        ]
 [  0.           2.33333333   0.33333333   0.           1.
   -0.33333333   5.        ]
 [  0.           0.66666667   1.66666667   1.           0.
    0.33333333   5.        ]]


{'Z': -20.0, 'coeficientes': [0, 0, 5.0, 5.0, 0]}